In [ ]:
import tweepy
from textblob import TextBlob

con_key = 'XXXX'
con_secret = 'XXXX'
acc_token = 'XXXX'
acc_secret = 'XXX'

#Use tweepy.OAuthHandler to create an authentication using the given key and secret
auth = tweepy.OAuthHandler(consumer_key=con_key, consumer_secret=con_secret)
auth.set_access_token(acc_token, acc_secret)

#Connect to the Twitter API using the authentication
api = tweepy.API(auth)

#Use the REST API for two static searches to find recent tweets between Dec. 8-10, using the hashtags #metoo and #blacklivesmatter
#each search removed retweets and replies to get original tweets, gathered the full tweet using tweet_mode='extended' as most were truncated
#only tweets in english were analyzed

num_needed = 1000
metoo_list = []
last_id = -1 # id of last tweet seen
while len(metoo_list) < num_needed:
    try: 
        metoo_tweets = api.search(q = 'since:2017-12-8 until:2017-12-10 #%23metoo -filter:retweets -filter:replies', tweet_mode='extended', lang = 'en', count = 1000, max_id = str(last_id - 1))
    except tweepy.TweepError as e:
        print("Error", e)
        break
    else:
        if not metoo_tweets:
            print("Could not find any more metoo tweets!")
            break
        metoo_list.extend(metoo_tweets)
        last_id = metoo_tweets[-1].id

blm_list = []
while len(blm_list) < num_needed:
    try:
        blm_tweets = api.search(q = 'since:2017-12-8 until:2017-12-10 #%23blacklivesmatter -filter:retweets -filter:replies', tweet_mode='extended', lang = 'en', count = 1000, max_id = str(last_id - 1))
    except tweepy.TweepError as e:
        print("Error", e)
        break
    else:
        if not blm_tweets:
            print("Could not find any more blm tweets!")
            break
        blm_list.extend(blm_tweets)
        last_id = blm_tweets[-1].id
print(len(metoo_list))
print(len(blm_list))

In [ ]:
import numpy as np
import pandas as pd

#here's my code which created a dataframe for the #metoo hashtag
metoo_data = pd.DataFrame([tweet.full_text for tweet in metoo_list], columns=['Tweets']) #placed all tweets into a column
#these four lines removed all URls, all incorrectly translated &'s and >'s, and the #metoo hashtag from all tweets
metoo_data['Tweets'] = metoo_data['Tweets'].str.replace('http\S+|www.\S+', '', case=False)
metoo_data['Tweets'] = metoo_data['Tweets'].str.replace('&amp;', '&', case=False)
metoo_data['Tweets'] = metoo_data['Tweets'].str.replace('&gt;', '>', case=False)
metoo_data['Tweets'] = metoo_data['Tweets'].str.replace('#metoo', '', case=False)
#these three lines created columns for and counted favorites, retweets; then added favorites and retweets into another column
metoo_data['Favorite'] = np.array([tweet.favorite_count for tweet in metoo_list])
metoo_data['Retweets'] = np.array([tweet.retweet_count for tweet in metoo_list])
metoo_data['Engagement'] = metoo_data['Favorite'] + metoo_data['Retweets']
#evaluated Sentiment, specifically polarity, using the TextBlob package
metoo_data['Sentiment'] = metoo_data['Tweets'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
#counted the number of characters for each tweet and placed into another column
metoo_data['Tweet Length'] = metoo_data['Tweets'].str.len()
#these three lines counted the number of exclamation points, question marks and capital letters; then added those into aother column
metoo_data['Exclamations/Questions'] = metoo_data['Tweets'].str.count("[!?]")
metoo_data['Capital Letters'] = metoo_data['Tweets'].str.count("[A-Z]")
metoo_data['Passion'] = metoo_data['Exclamations/Questions'] + metoo_data['Capital Letters']
#display the #metoo dataframe
metoo_data

In [ ]:
#here's my code which created a dataframe for the #blacklivesmatter hashtag
blm_data = pd.DataFrame([tweet.full_text for tweet in blm_list], columns=['Tweets'])
#these four lines removed all URls, all incorrectly translated &'s and >'s, and the #blacklivesmatter hashtag from all tweets
blm_data['Tweets'] = blm_data['Tweets'].str.replace('http\S+|www.\S+', '', case=False)
blm_data['Tweets'] = blm_data['Tweets'].str.replace('&amp;', '&', case=False)
blm_data['Tweets'] = blm_data['Tweets'].str.replace('&gt;', '>', case=False)
blm_data['Tweets'] = blm_data['Tweets'].str.replace('#blacklivesmatter', '', case=False)
#these three lines created columns for and counted favorites, retweets; then added favorites and retweets into another column
blm_data['Favorite'] = np.array([tweet.favorite_count for tweet in blm_list])
blm_data['Retweets'] = np.array([tweet.retweet_count for tweet in blm_list])
blm_data['Engagement'] = blm_data['Favorite'] + blm_data['Retweets']
#evaluated Sentiment, specifically polarity, using the TextBlob package
blm_data['Sentiment'] = blm_data['Tweets'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
#counted the number of characters for each tweet and placed into another column
blm_data['Tweet Length'] = blm_data['Tweets'].str.len()
#these three lines counted the number of exclamation points, question marks and capital letters; then added those into aother column
blm_data['Exclamations/Questions'] = blm_data['Tweets'].str.count("[!?]")
blm_data['Capital Letters'] = blm_data['Tweets'].str.count("[A-Z]")
blm_data['Passion'] = blm_data['Exclamations/Questions'] + blm_data['Capital Letters']
#display the #blacklivesmatter dataframe
blm_data

In [ ]:
#writes #metoo dataframe into a csv file
metoo_data.to_csv("metoo_data4.csv", index=False)

In [ ]:
#writes #blacklivesmatter dataframe into a csv file
blm_data.to_csv("blm_data4.csv", index=False)